In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import numpy as np
import cv2
from tensorflow.keras.models import load_model
from sklearn.preprocessing import MaxAbsScaler
import pandas as pd
import os
from PIL import Image
import re
import matplotlib.pyplot as plt
import numpy as np
import pickle
from dotenv import load_dotenv

load_dotenv()

from pydrive2.auth import GoogleAuth
from pydrive2.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
# keras_model = load_model('/content/drive/MyDrive/Skripsi/model/model_2_fast.h5')
%matplotlib inline


Mounted at /content/drive


# Comparison Test

The purpose of comparison test is to calculate the difference between real carbon value from lab and predicted value using proposed model to test how good the model is when predicting original data. 

In [ ]:
path_csv_fase0 = '/content/drive/MyDrive/Skripsi/Karbon/fase0_carbon_lab.csv'
path_csv_fase1 = '/content/drive/MyDrive/Skripsi/Karbon/fase1_carbon_lab.csv'
path_csv_fase2 = '/content/drive/MyDrive/Skripsi/Karbon/fase2_carbon_lab.csv'



load_hst0_100m = "/content/drive/MyDrive/Skripsi/Cropped_Image/Kowang_Fase_0/100m/30X30/image_source"
load_hst0_50m = "/content/drive/MyDrive/Skripsi/Cropped_Image/Kowang_Fase_0/50m/30X30/image_source"
load_hst0_25m = "/content/drive/MyDrive/Skripsi/Cropped_Image/Kowang_Fase_0/25m/30X30/image_source"

load_hst1_100m = "/content/drive/MyDrive/Skripsi/Cropped_Image/Tulung_Fase_1/100m/30X30/image_source"
load_hst1_50m = "/content/drive/MyDrive/Skripsi/Cropped_Image/Tulung_Fase_1/50m/30X30/image_source"
load_hst1_25m = "/content/drive/MyDrive/Skripsi/Cropped_Image/Tulung_Fase_1/25m/30X30/image_source"

load_hst2_100m = "/content/drive/MyDrive/Skripsi/Cropped_Image/Kowang_Fase_2/100m/30X30/image_source"
load_hst2_50m = "/content/drive/MyDrive/Skripsi/Cropped_Image/Kowang_Fase_2/50m/30X30/image_source"
load_hst2_25m = "/content/drive/MyDrive/Skripsi/Cropped_Image/Kowang_Fase_2/25m/30X30/image_source"



In [ ]:
def convert_tif_to_png(directory, pattern=r'\w+', size=None):
    # List all files in the directory
    files = os.listdir(directory)

    # Loop through all files in the directory
    for file in files:
        # Check if the file is a .tif file
        if file.endswith('.tif') and re.match(pattern, file):
            # Construct the full file path
            old_file = os.path.join(directory, file)

            # Open the .tif file
            with Image.open(old_file) as img:
                # Resize if size is specified
                if size:
                    img = img.resize(size, Image.ANTIALIAS)

                # Construct the new file path with .png extension
                new_file = os.path.join(directory, file.replace('.tif', '.png'))

                # Save the image as .png
                img.save(new_file, 'PNG')

    print(os.listdir(directory))


def features_extraction_and_descriptors(image):
    df = pd.DataFrame(columns=['Feature Extraction', 'Descriptors'])

    img = cv2.imread(image, cv2.IMREAD_GRAYSCALE)

    height, width = img.shape

    center_x, center_y = width // 2, height // 2

    # Calculate the starting and ending points of the crop
    start_x = max(center_x - 15, 0)
    start_y = max(center_y - 15, 0)
    end_x = start_x + 30
    end_y = start_y + 30

    # Make sure the crop dimensions are within the image bounds
    end_x = min(end_x, width)
    end_y = min(end_y, height)

    # Crop the image
    cropped_image = img[start_y:end_y, start_x:end_x]
    # print(cropped_image.shape)

    # Apply CLAHE
    clahe = cv2.createCLAHE(clipLimit=1.0, tileGridSize=(10, 10))
    cl_img = clahe.apply(cropped_image)
    sift = cv2.SIFT_create()


    # Detect FAST keypoints
    fast = cv2.FastFeatureDetector_create()
    img_kp = fast.detect(cl_img)
    # img_kp = sift.detect(cl_img)
    pts_new = [img_kp[x].pt for x in range(len(img_kp))]
    np_newlist = np.array(pts_new, dtype=np.float32).tolist()

    # Compute SIFT descriptors

    kp, des = sift.compute(cl_img, img_kp, descriptors=np.array([]))

    # Convert descriptors to a list
    sift_descriptors = np.array(des,dtype=np.float32).tolist()

    df = df._append(pd.Series([np_newlist,sift_descriptors], index = ['Feature Extraction', 'Descriptors']), ignore_index = True )

    return df

def predict_carbon(image):
  scaler_filename = 'scaler.pkl'

  with open('/content/drive/MyDrive/Skripsi/model/' + scaler_filename, 'rb') as scaler_file:
    scaler = pickle.load(scaler_file)

  df = features_extraction_and_descriptors(image)
  df['Feature Extraction'] = df['Feature Extraction'].apply(lambda x: [item for sublist in x for item in sublist])
  df['Descriptors'] = df['Descriptors'].apply(lambda x: [item for sublist in x for item in sublist])


  df_new = pd.DataFrame()

  for index, row in df.iterrows():
      # Extract 'Feature Extraction' and 'Descriptors'
      feature_extraction = row['Feature Extraction']
      descriptors = row['Descriptors']

      max_length = max(len(feature_extraction), len(descriptors))

      padded_feature_extraction = np.pad(feature_extraction, (0, max_length - len(feature_extraction) ), mode='constant')

      padded_descriptors = np.pad(descriptors, (0, max_length - len(descriptors)), mode='constant')

      feature_extraction_cols = [f'Feature Extraction {i}' for i in range(max_length)]
      descriptors_cols = [f'Descriptors {i}' for i in range(max_length)]

      new_row_data = {
          **{col: val for col, val in zip(feature_extraction_cols, padded_feature_extraction)},
          **{col: val for col, val in zip(descriptors_cols, padded_descriptors)}
      }

      df_new = pd.concat([df_new, pd.DataFrame(new_row_data, index=[0])], ignore_index=True)

  value_df = df_new

  # Flatten the dataframe
  value_df_flattened = np.array([x.flatten() for x in value_df.values])

  value_df_normalized = scaler.fit_transform(value_df_flattened)

  # Reshape to match the model input shape
  value_df_reshaped = value_df_normalized.reshape((value_df_normalized.shape[0], value_df_normalized.shape[1], 1))


  keras_model = load_model('/content/drive/MyDrive/Skripsi/model/model_fast_v11.h5')

  prediction = keras_model.predict(value_df_reshaped)

  carbon = float(prediction[0][0])

  carbon = round(prediction[0][0], 2)

  return carbon


def extract_carbon_values(csv_path):
    # Read the CSV file into a pandas DataFrame
    df = pd.read_csv(csv_path,encoding='cp1252',delimiter=';', converters={'carbon_value': lambda x: float(x.replace(',', '.'))})

    # Assuming the column containing carbon values is named 'CarbonValue'
    carbon_values = df['carbon_value'].tolist()

    return carbon_values

def calculate_carbon_difference(path_csv, image_directory):

  predicted_list = predict_carbon_for_directory(image_directory)

  actual_list = extract_carbon_values(path_csv)

  differences = [abs(predicted_list[i] - actual_list[i]) for i in range(len(predicted_list))]

  # Print or use the differences list
  return differences


def get_image_names(image_directory):
    image_names = []

    # List files in the directory
    files = os.listdir(image_directory)

    # Extract file names (assuming they are image files like PNG or JPEG)
    for file in files:
        if file.lower().endswith(('.png')):
            image_names.append(file)


    # Sort the file names based on their numeric values
    sorted_image_names = sorted(image_names, key=lambda x: int(x.split('.')[0]))

    return sorted_image_names

def predict_carbon_for_directory(directory):
    predictions = []

    # Iterate over files in the directory

    image_names = get_image_names(directory)

    print(image_names)

    for file in image_names:
        file_path = os.path.join(directory, file)

        # Check if the file is an image (you may want to refine the check based on your image types)
        if file.endswith('.png'):
            # Predict carbon value for each image
            carbon_prediction = predict_carbon(file_path)

            # Store the prediction along with the file name or any other relevant information
            predictions.append(carbon_prediction)

    return predictions


def construct_df(path_csv, image_directory):

  predicted_list = predict_carbon_for_directory(image_directory)

  actual_list = extract_carbon_values(path_csv)

  difference = calculate_carbon_difference(path_csv, image_directory)

  img_name = get_image_names(image_directory)


  df = pd.DataFrame({
        'Image Name': img_name,
        'Actual Carbon Value': actual_list,
        'Predicted Carbon Value': predicted_list,
        'Difference': difference
    })

  df_rounded = df.round(2)
  return df_rounded


def save_to_excel(df, file_name):
  file_name = file_name + '.xlsx'
  df.to_excel(file_name, index=False)
  file_path = '/content/' + file_name
  folder_id = os.getenv("COMPARISON_DATA")
  file = drive.CreateFile({'title': file_name, 'parents': [{'id': folder_id}]})
  file.SetContentFile(file_path)
  file.Upload()

  print(f"Data saved to {file_name}")


In [ ]:
# # convert_tif_to_png(load_hst1_100m)
# convert_tif_to_png(load_hst1_50m)
# convert_tif_to_png(load_hst1_25m)
# convert_tif_to_png(load_hst2_100m)
# convert_tif_to_png(load_hst2_50m)
# convert_tif_to_png(load_hst2_25m)

['1.tif', '2.tif', '3.tif', '4.tif', '5.tif', '6.tif', '7.tif', '8.tif', '9.tif', '10.tif', '11.tif', '12.tif', '13.tif', '14.tif', '15.tif', '16.tif', '17.tif', '18.tif', '19.tif', '20.tif', '21.tif', '22.tif', '23.tif', '24.tif', '25.tif', '26.tif', '27.tif', '28.tif', '29.tif', '30.tif', '1.png', '2.png', '3.png', '4.png', '5.png', '6.png', '7.png', '8.png', '9.png', '10.png', '11.png', '12.png', '13.png', '14.png', '15.png', '16.png', '17.png', '18.png', '19.png', '20.png', '21.png', '22.png', '23.png', '24.png', '25.png', '26.png', '27.png', '28.png', '29.png', '30.png']
['1.tif', '2.tif', '3.tif', '4.tif', '5.tif', '6.tif', '7.tif', '8.tif', '9.tif', '10.tif', '11.tif', '12.tif', '13.tif', '14.tif', '15.tif', '16.tif', '17.tif', '18.tif', '19.tif', '20.tif', '21.tif', '22.tif', '23.tif', '24.tif', '25.tif', '26.tif', '27.tif', '28.tif', '29.tif', '30.tif', '1.png', '2.png', '3.png', '4.png', '5.png', '6.png', '7.png', '8.png', '9.png', '10.png', '11.png', '12.png', '13.png', '14.

In [ ]:
save_to_excel(construct_df(path_csv_fase0, load_hst0_100m), 'fase0_100m')
save_to_excel(construct_df(path_csv_fase0, load_hst0_50m), 'fase0_50m')
save_to_excel(construct_df(path_csv_fase0, load_hst0_25m), 'fase0_25m')

save_to_excel(construct_df(path_csv_fase1, load_hst1_100m), 'fase1_100m')
save_to_excel(construct_df(path_csv_fase1, load_hst1_50m), 'fase1_50m')
save_to_excel(construct_df(path_csv_fase1, load_hst1_25m), 'fase1_25m')

save_to_excel(construct_df(path_csv_fase2, load_hst2_100m), 'fase2_100m')
save_to_excel(construct_df(path_csv_fase2, load_hst2_50m), 'fase2_50m')
save_to_excel(construct_df(path_csv_fase2, load_hst2_25m), 'fase2_25m')

['1.png', '2.png', '3.png', '4.png', '5.png', '6.png', '7.png', '8.png', '9.png', '10.png', '11.png', '12.png', '13.png', '14.png', '15.png', '16.png', '17.png', '18.png', '19.png', '20.png', '21.png', '22.png', '23.png', '24.png', '25.png', '26.png', '27.png', '28.png', '29.png', '30.png']
1/1 [==============================] - 0s 227ms/step


1/1 [==============================] - 0s 211ms/step


1/1 [==============================] - 0s 317ms/step
['1.png', '2.png', '3.png', '4.png', '5.png', '6.png', '7.png', '8.png', '9.png', '10.png', '11.png', '12.png', '13.png', '14.png', '15.png', '16.png', '17.png', '18.png', '19.png', '20.png', '21.png', '22.png', '23.png', '24.png', '25.png', '26.png', '27.png', '28.png', '29.png', '30.png']
1/1 [==============================] - 0s 310ms/step
Data saved to fase0_100m.xlsx
['1.png', '2.png', '3.png', '4.png', '5.png', '6.png', '7.png', '8.png', '9.png', '10.png', '11.png', '12.png', '13.png', '14.png', '15.png', '16.png', '17.png', '18.png', '19.png', '20.png', '21.png', '22.png', '23.png', '24.png', '25.png', '26.png', '27.png', '28.png', '29.png', '30.png']
1/1 [==============================] - 0s 346ms/step
['1.png', '2.png', '3.png', '4.png', '5.png', '6.png', '7.png', '8.png', '9.png', '10.png', '11.png', '12.png', '13.png', '14.png', '15.png', '16.png', '17.png', '18.png', '19.png', '20.png', '21.png', '22.png', '23.png', '24.p

# Tensor model deploy

The purpose of this section is to try deploying the model before implement it in AWS Lambda.

In [ ]:
df = pd.DataFrame(columns=['Feature Extraction', 'Descriptors'])

# Read and preprocess the img
img = cv2.imread("/content/16-2.png", cv2.IMREAD_GRAYSCALE)


height, width = img.shape

center_x, center_y = width // 2, height // 2

# Calculate the starting and ending points of the crop
start_x = max(center_x - 15, 0)
start_y = max(center_y - 15, 0)
end_x = start_x + 30
end_y = start_y + 30

# Make sure the crop dimensions are within the image bounds
end_x = min(end_x, width)
end_y = min(end_y, height)

# Crop the image
cropped_image = img[start_y:end_y, start_x:end_x]
print(cropped_image.shape)

# Apply CLAHE
clahe = cv2.createCLAHE(clipLimit=1.0, tileGridSize=(10, 10))
cl_img = clahe.apply(cropped_image)
sift = cv2.SIFT_create()


# Detect FAST keypoints
fast = cv2.FastFeatureDetector_create()
img_kp = fast.detect(cl_img)
# img_kp = sift.detect(cl_img)
pts_new = [img_kp[x].pt for x in range(len(img_kp))]
np_newlist = np.array(pts_new, dtype=np.float32).tolist()

print(np_newlist)

# Compute SIFT descriptors

kp, des = sift.compute(cl_img, img_kp, descriptors=np.array([]))

# Convert descriptors to a list
sift_descriptors = np.array(des,dtype=np.float32).tolist()
print(len(sift_descriptors))

(30, 30)
[[9.0, 3.0], [16.0, 3.0], [3.0, 4.0], [12.0, 4.0], [26.0, 4.0], [17.0, 5.0], [23.0, 5.0], [5.0, 6.0], [14.0, 6.0], [21.0, 7.0], [26.0, 7.0], [4.0, 8.0], [15.0, 8.0], [19.0, 8.0], [8.0, 9.0], [25.0, 9.0], [3.0, 10.0], [11.0, 10.0], [8.0, 11.0], [14.0, 11.0], [4.0, 12.0], [19.0, 12.0], [6.0, 13.0], [26.0, 13.0], [3.0, 14.0], [8.0, 14.0], [13.0, 14.0], [16.0, 14.0], [22.0, 14.0], [22.0, 16.0], [7.0, 18.0], [16.0, 18.0], [19.0, 19.0], [24.0, 19.0], [12.0, 20.0], [6.0, 21.0], [3.0, 22.0], [11.0, 22.0], [21.0, 22.0], [26.0, 22.0], [5.0, 23.0], [9.0, 23.0], [16.0, 23.0], [12.0, 24.0], [19.0, 24.0], [3.0, 25.0], [5.0, 25.0], [8.0, 25.0], [13.0, 26.0], [18.0, 26.0], [23.0, 26.0], [25.0, 26.0]]
52


In [ ]:
df = df._append(pd.Series([np_newlist,sift_descriptors], index = ['Feature Extraction', 'Descriptors']), ignore_index = True )

df

,Feature Extraction,Descriptors
0,"[[9.0, 3.0], [16.0, 3.0], [3.0, 4.0], [12.0, 4...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."


In [ ]:
df['Feature Extraction'] = df['Feature Extraction'].apply(lambda x: [item for sublist in x for item in sublist])
df['Descriptors'] = df['Descriptors'].apply(lambda x: [item for sublist in x for item in sublist])

In [ ]:
df_new = pd.DataFrame()

for index, row in df.iterrows():
    # Extract 'Feature Extraction' and 'Descriptors'
    feature_extraction = row['Feature Extraction']
    descriptors = row['Descriptors']

    print(len(feature_extraction))
    print(len(descriptors))

    max_length = max(len(feature_extraction), len(descriptors))
    # max_length_feature = len(feature_extraction)
    # max_length_descriptor =  len(descriptors)
    padded_feature_extraction = np.pad(feature_extraction, (0, max_length - len(feature_extraction) ), mode='constant')

    padded_descriptors = np.pad(descriptors, (0, max_length - len(descriptors)), mode='constant')

    # feature_extraction_cols = [f'Feature Extraction {i}' for i in range(max_length_feature)]
    # descriptors_cols = [f'Descriptors {i}' for i in range(max_length_descriptor)]

    feature_extraction_cols = [f'Feature Extraction {i}' for i in range(max_length)]
    descriptors_cols = [f'Descriptors {i}' for i in range(max_length)]

    new_row_data = {
        **{col: val for col, val in zip(feature_extraction_cols, padded_feature_extraction)},
        **{col: val for col, val in zip(descriptors_cols, padded_descriptors)}
    }

    df_new = pd.concat([df_new, pd.DataFrame(new_row_data, index=[0])], ignore_index=True)

104
6656


In [ ]:
value_df = df_new
value_df

,Feature Extraction 0,Feature Extraction 1,Feature Extraction 2,Feature Extraction 3,Feature Extraction 4,Feature Extraction 5,Feature Extraction 6,Feature Extraction 7,Feature Extraction 8,Feature Extraction 9,...,Descriptors 6646,Descriptors 6647,Descriptors 6648,Descriptors 6649,Descriptors 6650,Descriptors 6651,Descriptors 6652,Descriptors 6653,Descriptors 6654,Descriptors 6655
0,9.0,3.0,16.0,3.0,3.0,4.0,12.0,4.0,26.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:

value_df = value_df.values.reshape((value_df.shape[0],value_df.shape[1], 1))
print(value_df.shape)

(1, 13312, 1)


In [ ]:
drive.mount('/content/drive')

keras_model_1 = load_model('/content/drive/MyDrive/Skripsi/model/model_fast_v4.h5')
keras_model_2 = load_model('/content/drive/MyDrive/Skripsi/model/model_2_sift.h5')



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
keras_model_1.summary()

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, None, 1)]         0         
                                                                 
 conv1d_18 (Conv1D)          (None, None, 16)          64        
                                                                 
 batch_normalization_18 (Ba  (None, None, 16)          64        
 tchNormalization)                                               
                                                                 
 max_pooling1d_12 (MaxPooli  (None, None, 16)          0         
 ng1D)                                                           
                                                                 
 conv1d_19 (Conv1D)          (None, None, 32)          1568      
                                                                 
 batch_normalization_19 (Ba  (None, None, 32)          128 

In [ ]:
keras_model_2.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, None, 1)]         0         
                                                                 
 conv1d_9 (Conv1D)           (None, None, 16)          64        
                                                                 
 batch_normalization_9 (Bat  (None, None, 16)          64        
 chNormalization)                                                
                                                                 
 max_pooling1d_7 (MaxPoolin  (None, None, 16)          0         
 g1D)                                                            
                                                                 
 conv1d_10 (Conv1D)          (None, None, 32)          1568      
                                                                 
 batch_normalization_10 (Ba  (None, None, 32)          128 

In [ ]:
prediction = keras_model_1.predict(value_df)
# predictions_df = pd.DataFrame(prediction, columns=['Predicted Value'])
# df_with_predictions = pd.concat([df, predictions_df], axis=1)

print(prediction[0][0])

carbon = float(prediction[0][0])

carbon = round(prediction[0][0], 2)


1/1 [==============================] - 0s 188ms/step
54.016777


In [ ]:
def coba():
  return {
      "statusCode" : 200,
      "body" : {
         "carbon" : carbon
          }
  }


In [ ]:
coba()

{'statusCode': 200, 'body': {'carbon': 54.02}}